## 11.5 从序列到序列

到目前为止,我们已经接触过了大多数 nlp 任务所需要的工具,然而我们只在处理过一类 nlp 任务--文本分类,这一节我们将接触另一种序列到序列的模型.

序列到序列模型是将一个序列作为输入(可以是一个句子或一段文字)并将其翻译成不同的序列,这是许多 nlp 的核心任务.

- 机器翻译: 将源样本的段落转换为目标语言的对应段落.
- 文本汇总: 将长文本进行总结,只保留最重要信息.
- 问题问答: 输入问题,返回答案
- 聊天机器人: 输入对话/历史聊天记录等等转换为下一个回复
- 文本生成: 将输入文本转换为一个完整的段落.
- 等等...


![seq2seq](seq2seq.png)

上图显示了序列到序列模型工作的一般流程

训练期

- 一个编码器将源序列转换为一种中间表示.
- 通过查看之前的标记和源序列,解码器被用来预测下一个标记.

预测,预测时我们无法接触到目标序列,我们不得不一次生成一个标记.

- 编码器将源序列编码转换
- 解码器查看源序列和初始'种子'标记(图中是 '[start]'),并预测下一个真实标记.
- 将真实标记送入解码器推动下一次预测,直到生成一个停止标记(图中是 '[end]').


## 一个机器翻译例子

我们将在一个机器翻译的问题上演示序列到序列的模型.机器翻译正是 Transformer 开发的目的,从一个递归的序列模型开始,我们最终会用到完整的 Transformer 模型.


In [1]:
# !wget http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
# !unzip -q spa-eng.zip

英语到西班牙语的样本数据.

- 文本的每一行都是一个例子
- 英语 制表符 西班牙语


In [2]:
text_file = "spa-eng/spa.txt"
with open(text_file, encoding='UTF-8') as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    english, spanish = line.split("\t")
    spanish = "[start] " + spanish + " [end]"
    text_pairs.append((english, spanish))

In [3]:
import random
print(random.choice(text_pairs))

("It's wrong to deceive people, but worse to deceive yourself.", '[start] Está mal engañar a la gente, pero es aún peor engañarte a ti mismo. [end]')


一个样本示例


In [4]:
import random

random.shuffle(text_pairs)  #随机排序
num_val_samples = int(0.15 * len(text_pairs))  #15%的验证数据
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]  #70%的训练数据
val_pairs = text_pairs[num_train_samples:num_train_samples +
                       num_val_samples]  #15%的验证数据
test_pairs = text_pairs[num_train_samples + num_val_samples:]  #15%的测试数据


将整个样本分割: 70:15:15 训练:验证:测试


In [5]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import tensorflow as tf
import string
import re

strip_chars = string.punctuation + "¿"  # strip_chars = string.punctuation(所有标点符号) + ¿
strip_chars = strip_chars.replace("[", "")  #不替换 [
strip_chars = strip_chars.replace("]", "")  #不替换 ]


def custom_standardization(input_string):  #全部小写,删除除 '[' ']' 以外的全部标点
    lowercase = tf.strings.lower(input_string)  #小写
    #                 按照正则表达式替换          格式化字符串 re.escape 自动添加转义
    return tf.strings.regex_replace(lowercase, f"[{re.escape(strip_chars)}]",
                                    "")


vocab_size = 15000  #设置最大词汇量
sequence_length = 20  #设置最大句子长度

source_vectorization = TextVectorization(  #英语
    max_tokens=vocab_size,  #最大的词汇量
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = TextVectorization(  #西班牙语
    max_tokens=vocab_size,  #最大的词汇量
    output_mode="int",
    output_sequence_length=sequence_length +
    1,  #生成具有额外标记的西班牙语句子,我们需要在训练时将句子偏转一步(进行预测时 [start] 和 [end] 的缘故)
    standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_spanish_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)  #学习
target_vectorization.adapt(train_spanish_texts)

因为语言差别,西班牙语和英语需要互相独立处理.西班牙语处理要传入自定义的字符串格式化函数.

- 保留 '[' 和 ']',默认情况下 '[' 和 ']' 都会被删除,但是为了区分 `start` 和 `[start]` 我们需要保留 '[' 和 ']'.(这里是仅西班牙语中有)
- 不同语言的标点符号有差别,如果英语中我们选择去掉标点符号,那么对应的西班牙语中的 `¿` 也需要删除.

**警告**: 对于应用在生产环境的模型,一般应该将标点符号映射为独立的标记,而不是直接删除.如果直接删除,那输出译文不会有正确的标点符号.在我们的例子中仅仅是为了简化代码而删除了标点符号.


In [6]:
batch_size = 64  #批次大小


def format_dataset(eng, spa):
    eng = source_vectorization(eng)  #英文 转换为向量
    spa = target_vectorization(spa)  #西班牙文 转换为向量
    return (
        {
            "english": eng,
            "spanish": spa[:, :-1],  #输入西班牙语句子 不包括最好一个字符,保证输入与目标相同的长度
        },
        spa[:, 1:])  #目标西班牙语句子 提前一步, 输入/输出依然是相同的长度.


def make_dataset(pairs):  #构造数据集
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.shuffle(2048).prefetch(16).cache()


train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

数据集是一个元组 (inputs, target),inputs 是字典,两个 key `encoder_inputs`(英语) 和 `decoder_inputs`(西班牙语)

In [7]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['spanish'].shape: {inputs['spanish'].shape}")
    print(f"targets.shape: {targets.shape}")

inputs['english'].shape: (64, 20)
inputs['spanish'].shape: (64, 20)
targets.shape: (64, 20)


至此数据准备完毕.


## RNN 序列到序列模型

RNN 在 2015~2017 年主导了序列到序列模型的开发,然后其地位被 Transformer 取代.当时 RNN 是许多现实世界机器翻译系统的基础,例如 2017 年时 google 翻译就是由 7 个大型 ltsm 层堆叠的网络.但是今天这样的模型依旧值得学习,它提供了一个理解序列到序列模型的简单入口.


In [9]:
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

In [24]:
inputs = keras.Input(shape=(sequence_length, ), dtype="int64")
x = layers.Embedding(input_dim=vocab_size, output_dim=128)(inputs)
x = layers.LSTM(32, return_sequences=True)(x)  #LSTM 返回序列
outputs = layers.Dense(vocab_size, activation="softmax")(x)  #输出
model = keras.Model(inputs, outputs)

最直观的使用 rnn 的序列到序列模型就是每个迭代后,保持 rnn 的输出.但是这样做有几个问题

- 目标序列长度必须和源序列长度一致,这在实际条件下非常少见,但是我们随时可以对序列进行填充到两者长度一致,因此这不是关键问题.
- 由于 rnn 的特点,这样的模型预测目标序列 `0~N` 只关注源序列 `0~N` 的标记,则在翻译任务中无法适用.一个例子: 'he weather is nice today' 翻译为法语 'Il fait beau aujourd’hui'.模型需要仅通过  `The` 就能翻译出 `Il` ,通过 `The weather`  翻译出 `Il fait`,这几乎是不可能完成的.

如果是人类译者,应该都会在写下译文前通读整个句子,特别是处理的语言在语序上非常不同时(例如: 英文和日文).而这也是标准序列到序列模型需要做的事情.


![rnn_seq2seq](rnn_seq2seq.png)

如上图所示:

- 首先要使用编码器(RNN),将输入整个源序列转换成单一张量(或一组张量).
- 然后将这个张量(或者一组张量)传递给另外一个解码器(RNN),作为解码器的初始状态,它将查看目标序列的 0~N,并尝试预测目标序列到 N+1.


In [25]:
from tensorflow import keras
from tensorflow.keras import layers

embed_dim = 256  #词向量维度
latent_dim = 1024  #隐藏层维度

source = keras.Input(shape=(None, ), dtype="int64", name="english")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)
encoded_source = layers.Bidirectional(layers.GRU(latent_dim),
                                      merge_mode="sum")(x)


这里编码器的实现: 使用 gru 代替了 ltsm 因为 gru 更加简单,只有一个状态向量.


In [26]:
past_target = keras.Input(shape=(None, ), dtype="int64", name="spanish")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)  #词嵌入
decoder_gru = layers.GRU(latent_dim, return_sequences=True)
x = decoder_gru(x, initial_state=encoded_source)  #gru
x = layers.Dropout(0.5)(x)  #dropout 层
target_next_step = layers.Dense(vocab_size, activation="softmax")(
    x)  #密集层 每个输出步骤尝试西班牙语词的概率分布
seq2seq_rnn = keras.Model([source, past_target], target_next_step)

解码器实现: 一个简单的 gru 层,gru 层将编码后的英文序列作为初始状态,在 gru 之后又接入了一个密集层,为每个步骤产生一个西班牙语词汇的概率分布.

训练过程中,解码器将整个目标序列作为输入,但是由于 rnn 所以解码器是只看 0~N 预测标记 N,这个过程和前文处理时序数据时意义相同,即 rnn 中我们只能使用 '过去' 的数据预测 '未来',绝对不能打破这个过程,否则我们的模型无法完成翻译工作.


In [27]:
seq2seq_rnn.compile(optimizer="rmsprop",
                    loss="sparse_categorical_crossentropy",
                    metrics=["accuracy"])
seq2seq_rnn.fit(train_ds, epochs=15, validation_data=val_ds)

Epoch 1/15
1302/1302 [==============================] - 131s 94ms/step - loss: 1.6413 - accuracy: 0.4153 - val_loss: 1.3122 - val_accuracy: 0.5051
Epoch 2/15
1302/1302 [==============================] - 123s 95ms/step - loss: 1.3216 - accuracy: 0.5248 - val_loss: 1.1524 - val_accuracy: 0.5680
Epoch 3/15
1302/1302 [==============================] - 133s 102ms/step - loss: 1.1800 - accuracy: 0.5739 - val_loss: 1.0718 - val_accuracy: 0.5998
Epoch 4/15
1302/1302 [==============================] - 132s 101ms/step - loss: 1.0885 - accuracy: 0.6059 - val_loss: 1.0333 - val_accuracy: 0.6182
Epoch 5/15
1302/1302 [==============================] - 132s 101ms/step - loss: 1.0430 - accuracy: 0.6299 - val_loss: 1.0207 - val_accuracy: 0.6287
Epoch 6/15
1302/1302 [==============================] - 130s 100ms/step - loss: 1.0130 - accuracy: 0.6481 - val_loss: 1.0158 - val_accuracy: 0.6344
Epoch 7/15
1302/1302 [==============================] - 129s 99ms/step - loss: 0.9936 - accuracy: 0.6618 - val_los

这里看准确率是 64.4%: 64% 的时间内正确预测了西班牙语的下一个单词...

但是评价机器翻译,但是预测下一个词的正确率这个标准并不适合.特别是这个标准的一个假设前提: 预测前 N+1 个标记时,已经知道 0~N 的正确目标标记了.实际上在预测过程中,我们是从头生成目标序列,不能依赖以前生成标记是 100% 准确的.

在现实世界中考察机器翻译,一般会使用 BLEU 分数,这个是考察整个生成序列的指标,似乎与人类对翻译质量的感知正相关.


In [28]:
import numpy as np

spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20


def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])
        next_token_predictions = seq2seq_rnn.predict(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(next_token_predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence


test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):  #随机取了 20组
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
The enemy dropped bombs on the factory.
[start] el profesor se dijo en la guerra [end]
-
She had white shoes on.
[start] ella tenía los zapatos [UNK] [end]
-
The letter does not say what time she will come up to Tokyo.
[start] la carta no se va a decir que ella había ido a tokio [end]
-
We are having a mild winter.
[start] estamos en un año [end]
-
What are your responsibilities?
[start] cuál son tus cosas [end]
-
Tom fed the goats.
[start] tom de [UNK] a las [UNK] [end]
-
Tom has a pain in his big toe on his right foot.
[start] tom tiene un gran en su [UNK] en su gran estoy de la un todo el trabajo [end]
-
New Zealand is pretty incredible.
[start] [UNK] nueva es [UNK] [end]
-
I found a place to live.
[start] encontré un lugar para vivir [end]
-
They all left.
[start] todos se [UNK] [end]
-
Japan is not as large as Canada.
[start] japón no es tan grande como un aquí [end]
-
Tom isn't sure.
[start] tom no está seguro [end]
-
Tom is a thirteen-year-old boy.
[start] tom es un hombre de 

这里直接取了 20 组结果,看不懂西班牙文,所以无法人肉评价.书上作者的评价是作为一个玩具而言,效果还可以,尽管还有很多错误.

注意: 这里的代码非常简单,但是效率却很低,我们每次采样一个新词就会重新处理整个源序列和整个生成的目标序列,实际应用中会将编码器和解码器拆开成两个独立模型,解码器每次标记采样迭代时只运行一个步骤,重用其中的内部状态.


关于这个玩具模型的改进

- 编码器/解码器 堆叠更多的 rnn 层.
- 使用 ltsm 而不是 gru
- 等等

然而 rnn 的序列到序列模型有一些天然的劣势

- 源序列必须完全保存在编码器的状态向量中,这对模型能够翻译的句子大小和复杂性有非常大的限制.这有点像人类翻译句子仅凭记忆的知识,而不是多看几遍原句.
- rnn 在比较难以处理长序列.rnn 常常会遗忘过去.当你的序列长度超过 100 时,第 100 个标记输入模型,然而此时模型几乎没有序列一开始输入的信息了.这意味着 rnn 模型无法长时间保持上下文,这个缺点翻译长篇文章是致命的.

上面的限制是 2017 年以后业内从 rnn 大量转向 Transformer 的原因.


## Transformer 序列到序列模型

序列到序列的模型是 Transformer 真正能发挥作用的地方,Neural attention 使得 Transformer 模型能比 rnn 更成功处理更复杂/更长序列.

人肉将英语翻译为西班牙语时,我们不可能一个词一个词的读英文记住,再一个词一个词的翻译成西班牙语,这样翻译方式对一句话5个词,这样的文本还行,对付一大段文字就无能为力了.实际上人肉翻译时(比如现在,打下译文),我们会在译文和原句之间反复横跳,当写下译文时会参考原句/原句上下很多不同的词语构成的语境.

上面那样人肉翻译的模式,正是可以通过 neural attention 和 Transformer 实现的.上一节我们已经熟悉了 Transformer 的编码器,编码器使用 elf-attention 实现对输入序列上下文感知的表示.在 Transformer 实现的序列到序列的模型中,编码器自然还是编码器的角色,读取源序列并输出源序列的编码表示.与 rnn 模型不同的是,Transformer 编码器的输出依旧是序列(一串上下文感知的嵌入向量)

Transformer 序列到序列模型的第二个部分是 Transformer 解码器,如同 rnn 解码器,它会读取目标序列的 0~N 试图预测标记的 N+1.最重要的是在预测时候,解码器会通过 neural attention 识别源序列中哪些标记与目前试图预测标记最密切(或许还和人工翻译不那么一样).回顾一下 查询-键-值模型: Transformer 解码器中,目标序列作为了 查询用来关注源序列的不同部分(源序列承担了 键
 和 值).


![transformer](transformer.png)

上图完整显示了 Transformer 序列到序列模型,仔细看一下解码器内部,似乎与编码器非常相似,除了插入了额外的 self-attention 层.


### Transformer 解码器

与之前一样,我们要实现 Transformer 解码器,


In [10]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(num_heads=num_heads,
                                                     key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(num_heads=num_heads,
                                                     key_dim=embed_dim)
        self.dense_proj = keras.Sequential([
            layers.Dense(dense_dim, activation="relu"),
            layers.Dense(embed_dim),
        ])
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True  #这个属性确保了该层将输入序列的 mask 传播到输出上.

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(
            i >= j,
            dtype="int32")  #产生形状 （sequence_length, sequence_length）的 0-1 矩阵
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [
                tf.expand_dims(batch_size, -1),
                tf.constant([1, 1], dtype=tf.int32)
            ],
            axis=0
        )  #沿着批次轴复制它，得到一个形状矩阵（batch_size, sequence_length, sequence_length）。
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)  #获取 causal_mask
        if mask is not None:  #准备输入掩码
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)  #合并掩码
        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs,
            attention_mask=causal_mask)  # 因果掩码传递给 self-attention
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,  #将合并后的掩码传递给第二注意力层，该层将源序列与目标序列联系起来。
        )
        attention_output_2 = self.layernorm_2(attention_output_1 +
                                              attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)


值得说的是 `self.supports_masking = True`,keras 并没有默认启用 mask 支持,你无法将 mask 数组传递会没有实现 `compute_mask` 没有暴露 `supports_masking` 属性的层,这会导致一个报错.


```py
def get_causal_attention_mask(self, inputs):
    input_shape = tf.shape(inputs)
    batch_size, sequence_length = input_shape[0], input_shape[1]
    i = tf.range(sequence_length)[:, tf.newaxis]
    j = tf.range(sequence_length)
    mask = tf.cast(
        i >= j,
        dtype="int32")  #产生形状 （sequence_length, sequence_length）的 0-1 矩阵
    mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
    mult = tf.concat(
        [tf.expand_dims(batch_size, -1),
         tf.constant([1, 1], dtype=tf.int32)],
        axis=0
    )  #沿着批次轴复制它，得到一个形状矩阵（batch_size, sequence_length, sequence_length）。
    return tf.tile(mask, mult)
```


与 RNN 不同 Transformer 解码器解码器一次性看到全部输入.RNN 解码器每次只看一个步骤的输入,只能访问当前步骤的 0~N 预测当前步骤输出 N (目标序列的 N+1).Transformer 的解码器是不区分先后的,一次性看到整个目标序列,如果不加限制允许它使用整个输入,解码器最终会直接将输入的 N + 1 复制到输出 N 中,这样会使得模型准确度达到 100%.但是允许预测时,这样的模型是完全无用的,预测推理时,根本没有 N+1 的输入.

解决这个问题一个简单的方法: 屏蔽模型对此次迭代后续数据的访问,将输入顺序看作是时序后,屏蔽任何对未来信息的访问.

这个解决方案对应 `get_causal_attention_mask` 方法.


```py
def call(self, inputs, encoder_outputs, mask=None):
    causal_mask = self.get_causal_attention_mask(inputs)  #获取 causal_mask
    if mask is not None:  #准备输入掩码
        padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
        padding_mask = tf.minimum(padding_mask, causal_mask)  #合并掩码
    attention_output_1 = self.attention_1(
        query=inputs, value=inputs, key=inputs,
        attention_mask=causal_mask)  # 因果掩码传递给 self-attention
    attention_output_1 = self.layernorm_1(inputs + attention_output_1)
    attention_output_2 = self.attention_2(
        query=attention_output_1,
        value=encoder_outputs,
        key=encoder_outputs,
        attention_mask=padding_mask,  #将合并后的掩码传递给第二注意力层，该层将源序列与目标序列联系起来。
    )
    attention_output_2 = self.layernorm_2(attention_output_1 +
                                          attention_output_2)
    proj_output = self.dense_proj(attention_output_2)
    return self.layernorm_3(attention_output_2 + proj_output)
```


#### 组合



In [16]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim  #输入的维度
        self.dense_dim = dense_dim  #密集层大小
        self.num_heads = num_heads  #head 的数量
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)  #MultiHeadAttention 层
        self.dense_proj = keras.Sequential([
            layers.Dense(dense_dim, activation="relu"),
            layers.Dense(embed_dim),
        ])  #密集层
        self.layernorm_1 = layers.LayerNormalization()  #归一化层
        self.layernorm_2 = layers.LayerNormalization()  #归一化层

    def call(self, inputs, mask=None):  #实现 call 方法
        if mask is not None:  # 小细节: 词嵌入层生成的 mask 是二维的,但是 attention 层的期望是 3/4 维,这里要扩大 mask 维度
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):  #序列化,可以通过这个方法保存模型
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

In [17]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim,
                 **kwargs):  #位置嵌入缺点是必须事先知道序列长度
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(input_dim=input_dim,
                                                 output_dim=output_dim)  #嵌入层
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)  #标记位置嵌入层
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions  #两个嵌入层相加

    def compute_mask(
            self,
            inputs,
            mask=None):  #与直接使用 Embedding 一样,这个层应该能够生成 mask,这样就能忽略掉填充的数据,减少运算量.
        return tf.math.not_equal(inputs, 0)

    def get_config(self):  #保存模型时调用
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

In [18]:
embed_dim = 256
dense_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None, ), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)  #编码器

decoder_inputs = keras.Input(shape=(None, ), dtype="int64", name="spanish")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim,
                       num_heads)(x, encoder_outputs)  #对目标序列进行编码,将其与源结合
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)  #为输出位置预测
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [19]:
transformer.compile(optimizer="rmsprop",
                    loss="sparse_categorical_crossentropy",
                    metrics=["accuracy"])
transformer.fit(train_ds, epochs=30, validation_data=val_ds)

Epoch 1/30
1302/1302 [==============================] - 183s 137ms/step - loss: 1.6965 - accuracy: 0.4098 - val_loss: 1.3724 - val_accuracy: 0.4975
Epoch 2/30
1302/1302 [==============================] - 165s 127ms/step - loss: 1.3630 - accuracy: 0.5233 - val_loss: 1.1919 - val_accuracy: 0.5589
Epoch 3/30
1302/1302 [==============================] - 145s 112ms/step - loss: 1.2038 - accuracy: 0.5746 - val_loss: 1.1039 - val_accuracy: 0.5928
Epoch 4/30
1302/1302 [==============================] - 147s 113ms/step - loss: 1.1094 - accuracy: 0.6091 - val_loss: 1.0418 - val_accuracy: 0.6235
Epoch 5/30
1302/1302 [==============================] - 164s 126ms/step - loss: 1.0541 - accuracy: 0.6323 - val_loss: 1.0291 - val_accuracy: 0.6290
Epoch 6/30
1302/1302 [==============================] - 165s 127ms/step - loss: 1.0239 - accuracy: 0.6489 - val_loss: 1.0184 - val_accuracy: 0.6348
Epoch 7/30
1302/1302 [==============================] - 162s 124ms/step - loss: 0.9988 - accuracy: 0.6630 - val_

In [20]:
import numpy as np

spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20


def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence
                                                          ])[:, :-1]
        predictions = transformer(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence


test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
Is that so terrible?
[start] es eso tan terrible [end]
-
Tom and Mary are about the same height.
[start] tom y mary están de la mismo [UNK] [end]
-
Put out the light.
[start] [UNK] la luz [end]
-
I usually take a shower before breakfast.
[start] yo yo yo jamás yo no [UNK] [end]
-
The shop is closed today.
[start] la tienda está abierta hoy [end]
-
I don't like thick soup.
[start] no me gusta la sopa [UNK] [end]
-
How long have you been in Japan?
[start] cuánto tiempo estuviste en japón [end]
-
Tom sang to Mary.
[start] tom le compraste a mary [end]
-
Tom and Mary don't know what's happening.
[start] tom y mary no se [UNK] lo que está hablando [end]
-
She listened to music for hours.
[start] ella perdió la música por horas [end]
-
Tom and Mary drank beer together.
[start] tom y mary se [UNK] la cerveza juntos [end]
-
This bed is too hard to sleep in.
[start] esta cama es demasiado difícil para dormir en él [end]
-
He plays the violin very well.
[start] sabe bien la radio demasiado bie

这里就不评论效果了吧,毕竟我又看不懂西班牙语.


这一节真的是😵,希望 12 章好一点.
